In [2]:
import pandas as pd

# Pfad zur CSV-Datei
file_path = 'data/stations.csv'

# Laden der CSV-Datei
df = pd.read_csv(file_path)

# Ausgabe des DataFrame vor der Änderung
print("Vor der Änderung:")
print(df)

# Entfernen von Zeilen, in denen 'entityId' gleich 30011 ist
df = df[df['entityId'] != 30011]

# Alternativ mit drop und der Bedingung auf den index
# indices_to_drop = df[df['entityId'] == 30011].index
# df = df.drop(indices_to_drop)

# Ausgabe des DataFrame nach der Änderung
print("\nNach der Änderung:")
print(df)

# Optional: Speichern des geänderten DataFrame zurück in eine CSV-Datei
df.to_csv(file_path, index=False)

Vor der Änderung:
     entityId                    station_name  maximum_capacity   latitude  \
0       30434           Felde Gemeindezentrum                18  54.299552   
1       30207        Sporthalle Saxtorfer Weg                15  54.487005   
2       30151           Eiche Daenischenhagen                15  54.424663   
3       30147     Bushaltestelle Eiderbruecke                15  54.289706   
4       30146  Herzog Friedrich Strasse  ZOB                 25  54.317114   
..        ...                             ...               ...        ...   
190     24371               Anleger Reventlou                50  54.333647   
191     24370                     Alter Markt                25  54.322799   
192     24369         Alte Mu Brunswiker Str                 15  54.327583   
193     24368                       Umsteiger                22  54.314861   
194     24367           Anleger Dietrichsdorf                30  54.329611   

     longitude       subarea  
0     9.933817

In [1]:
import pandas as pd

# Pfad zur CSV-Datei
file_path = 'data/stations.csv'

# Laden der CSV-Datei
df = pd.read_csv(file_path)


In [2]:
df

,entityId,station_name,maximum_capacity,latitude,longitude,subarea
0,30434,Felde Gemeindezentrum,18,54.299552,9.933817,Felde
1,30207,Sporthalle Saxtorfer Weg,15,54.487005,9.835833,Eckernf√∂rde
2,30151,Eiche Daenischenhagen,15,54.424663,10.127872,Kiel
3,30147,Bushaltestelle Eiderbruecke,15,54.289706,10.090042,Kiel Umland
4,30146,Herzog Friedrich Strasse ZOB,25,54.317114,10.133099,Kiel
...,...,...,...,...,...,...
189,24371,Anleger Reventlou,50,54.333647,10.151714,Kiel
190,24370,Alter Markt,25,54.322799,10.139499,Kiel
191,24369,Alte Mu Brunswiker Str,15,54.327583,10.141500,Kiel
192,24368,Umsteiger,22,54.314861,10.131167,Kiel


In [3]:

df['subarea'] = df['subarea'].str.replace('√∂', 'ö')

In [4]:
df

,entityId,station_name,maximum_capacity,latitude,longitude,subarea
0,30434,Felde Gemeindezentrum,18,54.299552,9.933817,Felde
1,30207,Sporthalle Saxtorfer Weg,15,54.487005,9.835833,Eckernförde
2,30151,Eiche Daenischenhagen,15,54.424663,10.127872,Kiel
3,30147,Bushaltestelle Eiderbruecke,15,54.289706,10.090042,Kiel Umland
4,30146,Herzog Friedrich Strasse ZOB,25,54.317114,10.133099,Kiel
...,...,...,...,...,...,...
189,24371,Anleger Reventlou,50,54.333647,10.151714,Kiel
190,24370,Alter Markt,25,54.322799,10.139499,Kiel
191,24369,Alte Mu Brunswiker Str,15,54.327583,10.141500,Kiel
192,24368,Umsteiger,22,54.314861,10.131167,Kiel


In [5]:
# Speichern des Dataframe
df.to_csv('data/stations.csv', index=False)

***

In [5]:
import pandas as pd

# Pfad zur CSV-Datei
file_path = 'data/stations.csv'

# Laden der CSV-Datei
df = pd.read_csv(file_path)

In [6]:
df

,entityId,station_name,maximum_capacity,latitude,longitude,subarea,latitide
0,30434,Felde Gemeindezentrum,18,54.299552,9.933817,Felde,NaN
1,30207,Sporthalle Saxtorfer Weg,15,54.487005,9.835833,Eckernförde,NaN
2,30151,Eiche Daenischenhagen,15,54.424663,10.127872,Kiel,NaN
3,30147,Bushaltestelle Eiderbruecke,15,54.289706,10.090042,Kiel Umland,NaN
4,30146,Herzog Friedrich Strasse ZOB,25,54.317114,10.133099,Kiel,NaN
...,...,...,...,...,...,...,...
190,24370,Alter Markt,25,54.322799,10.139499,Kiel,NaN
191,24369,Alte Mu Brunswiker Str,15,54.327583,10.141500,Kiel,NaN
192,24368,Umsteiger,22,54.314861,10.131167,Kiel,NaN
193,24367,Anleger Dietrichsdorf,30,54.329611,10.178556,Kiel,NaN


In [7]:
df.drop('latitide', axis=1, inplace=True)


In [11]:
df = df[:-1]


In [8]:
df

,entityId,station_name,maximum_capacity,latitude,longitude,subarea
0,30434,Felde Gemeindezentrum,18,54.299552,9.933817,Felde
1,30207,Sporthalle Saxtorfer Weg,15,54.487005,9.835833,Eckernförde
2,30151,Eiche Daenischenhagen,15,54.424663,10.127872,Kiel
3,30147,Bushaltestelle Eiderbruecke,15,54.289706,10.090042,Kiel Umland
4,30146,Herzog Friedrich Strasse ZOB,25,54.317114,10.133099,Kiel
...,...,...,...,...,...,...
190,24370,Alter Markt,25,54.322799,10.139499,Kiel
191,24369,Alte Mu Brunswiker Str,15,54.327583,10.141500,Kiel
192,24368,Umsteiger,22,54.314861,10.131167,Kiel
193,24367,Anleger Dietrichsdorf,30,54.329611,10.178556,Kiel


In [13]:
# Erstellung einer neuen Zeile
new_entry = {
    'entityId': 99999,  # example ID
    'station_name': 'test',
    'maximum_capacity': 20,
    'latitude': 54.0944,
    'longitude': 10.2523, 
    'subarea': 'Plön',  # example subarea
    # Add other columns with example data if needed
}

# Hinzufügen der neuen Zeile zum DataFrame
# Convert the new_entry dictionary into a DataFrame and concatenate it to the original DataFrame
new_entry_df = pd.DataFrame([new_entry])


In [14]:
new_entry_df

,entityId,station_name,maximum_capacity,latitude,longitude,subarea
0,99999,test,20,54.0944,10.2523,Plön


In [ ]:

# Concatenate the dataframes
df = pd.concat([df, new_entry_df], ignore_index=True)



In [9]:

df.to_csv(file_path, index=False)

***

In [ ]:
import pandas as pd

# Pfad zur CSV-Datei

# Laden der CSV-Datei
df = pd.read_csv('data/stations.csv')